In [2]:
# Imports

import time, math
import numpy as np
from enum import Enum, auto

In [3]:
# Global Variables

max_weight = 5

file_logs = "Logs.txt"

class Deadlines(Enum):
    no_deadline = auto()
    date_deadline = auto()
    duration_deadline = auto()

class Repeats(Enum):
    endless = auto()
    till_date = auto()
    repeat_count = auto()

In [4]:
class Common_Functions:
    def log(self, message: str, display: bool) -> None:
        timestamp = time.strftime("%I:%M:%S %p, %d %b %Y")
        log_message = f"({timestamp}) {self.name}: {message}"

        with open(file_logs, "a") as file:
            file.write(log_message)

        if display:
            print(log_message)

    def log_error(self, error_type, error_message: str) -> None:
        self.log(error_message, False)
        raise error_type(error_message)

In [6]:
class Task(Common_Functions):
    task_number = 1 # Used to make the ID of the current task
    tasks_generated = 1 # Stores the number of tasks ever generated
    tasks_done = 0 # Stores the number of tasks ever done

    def __init__(self, name) -> None:
        self.name = name
        self.ID = self.task_number
        self.details = ""
        self.weight = 1
        self.deadline_type = Deadlines.no_deadline
        self.completed = False

        self.parent = None
        self.subtasks = np.array([])

        Task.task_number += 1
        Task.tasks_generated += 1

    def set_description(self, details: str) -> None:
        self.details = details
        self.log(f"Modified description")

    def set_priority(self, weight: float) -> None:
        if self.weight > max_weight:
            self.log_error(ValueError, f"Weight can't be greater than {max_weight}!")

        self.weight = weight
        self.log("Modified Weight")

    def set_deadline(self, deadline_type, deadline_info) -> None:
        match deadline_type:
            case Deadlines.date_deadline.value:
                self.deadline_type = Deadlines.date_deadline
            case Deadlines.time_deadline.value:
                self.deadline_type = Deadlines.time_deadline
            case Deadlines.duration_deadline.value:
                self.deadline_type = Deadlines.duration_deadline
            case Deadlines.repeat_deadline.value:
                self.deadline_type = Deadlines.date_deadline
            case _:
                self.deadline_type = Deadlines.no_deadline

    def set_subtask(self, subtask) -> None:
        if not isinstance(subtask, Task):
            self.log_error(TypeError, f"Can't make a non-Task object as sub-task!")

        self.subtasks = self.subtasks.append(self.subtasks, subtask)
        self.log(f"Added {subtask.name} as a subtask")

        subtask.parent = self
        subtask.log(f"Set {self.name} as a parent")

    def set_completion_status(self, status) -> None:
        self.completed = status
        self.log(f"Set completion status as {self.completed}")

        # Set subtasks to complete automatically as well
        # Check if parent's all substasks are completed

    def check_subtasks_completion(self) -> bool:
        if self.subtasks.size == 0:
            return False
        
        # "Vectorizes" the function which returns the completion status of subtasks
        # "Vectorization" of a function means that it can input an array of scalar instead of one scalar as usual (and thus outputs an array of scalars) 
        get_subtask_status = np.vectorize(lambda task: task.completed) # A function
        subtask_completions = get_subtask_status(self.subtasks)

        if np.all(subtask_completions == True):
            return True
        
        return False

# Make a provision for self completion status and parent completion status -
# If parent is completed - whole subtask is completed
# But if user sets the parent task uncomplete afterwards, set the completion status to what was self completion status
# Use the OR function between the self and parent completion status to achieve this

In [41]:
task1 = Task("Complete CS201 Quiz 1")
task1.show_maqsad()
task1.ID = 3
task1.show_maqsad()
task1.deadline_type

5
3


<Deadlines.no_deadline: 1>

In [40]:
task2 = Task("Go to Class - HS201")
task2.show_maqsad()
task1.ID = 10
task2.show_maqsad()
Task.ID = 1
task2.show_maqsad()

4
4
4
